# COMP5329 - Assignment 2

In [2]:
# import google
import collections
import json
import re

import nltk
import numpy as np
import pandas
import pandas as pd

# import torchtext
import PIL.Image
import timm
import torch
import torch.nn as nn
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MOUNT_PATH = '/content/drive'
# DRIVE_PATH = f'{MOUNT_PATH}/My Drive'
# PROJECT_PATH = DRIVE_PATH + "/Assignment 2"
PROJECT_PATH = "./"
IMG_PATH = f"{PROJECT_PATH}/data"
TRAIN_CSV_PATH = f"{PROJECT_PATH}/train.csv"
TEST_CSV_PATH = f"{PROJECT_PATH}/test.csv"

# google.colab.drive.mount(MOUNT_PATH)
torch.cuda.empty_cache()

In [2]:
# model = torch.load("efficientdet-d0.pth")

In [3]:
# from effdet import DetBenchTrain, EfficientDet, get_efficientdet_config
# from effdet.efficientdet import HeadNet


# def get_net():
#     config = get_efficientdet_config("tf_efficientdet_d0")
#     net = EfficientDet(config, pretrained_backbone=True)
#     #     checkpoint = torch.load('../input/efficientdet/efficientdet_d5-ef44aea8.pth')
#     #     net.load_state_dict(checkpoint)
#     num_outputs = 18
#     config.image_size = 224
#     net.class_net = HeadNet(config, num_outputs=18, norm_kwargs=dict(eps=0.001, momentum=0.01))
#     return DetBenchTrain(net, config)


# # model = get_net()

In [20]:
config = get_efficientdet_config("tf_efficientdet_d0")

config.image_size = [256, 256]
config.norm_kwargs = dict(eps=0.001, momentum=0.01)

model = EfficientDet(config, pretrained_backbone=True)
# checkpoint = torch.load('../input/efficientdet/efficientdet_d5-ef44aea8.pth')
# net.load_state_dict(checkpoint)

In [23]:
list(name for name, _ in model.named_children())

['backbone', 'fpn', 'class_net', 'box_net']

In [25]:
list(name for name, _ in model.named_children())

['backbone', 'fpn', 'class_net', 'box_net']

In [21]:
from torchinfo import summary

# img = torch.randn(1, 3, 224, 224)
# model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=18)
img = torch.randn(1, 3, 256, 256)
# summary(model, input_data=img, device='cpu')

In [ ]:
# from effdet import create_model

# model = create_model("tf_efficientdet_d0", bench_task="predict", num_classes=18, pretrained=True)

In [26]:
features = model.backbone(img)
for i in features:
    print(i.shape)

torch.Size([1, 40, 32, 32])
torch.Size([1, 112, 16, 16])
torch.Size([1, 320, 8, 8])


In [ ]:
40, 32, 32

In [27]:
fpn_out = model.fpn(features)
for i in fpn_out:
    print(i.shape)

torch.Size([1, 64, 32, 32])
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 8, 8])
torch.Size([1, 64, 4, 4])
torch.Size([1, 64, 2, 2])


In [29]:
class_out = model.class_net(fpn_out)
for i in class_out:
    print(i.shape)

torch.Size([1, 810, 32, 32])
torch.Size([1, 810, 16, 16])
torch.Size([1, 810, 8, 8])
torch.Size([1, 810, 4, 4])
torch.Size([1, 810, 2, 2])


In [30]:
model(img)

([tensor([[[[-4.3960, -4.3962, -4.2239,  ..., -4.1705, -4.3907, -4.4360],
            [-4.3503, -4.2365, -4.2250,  ..., -4.2904, -4.5557, -4.9295],
            [-4.5122, -4.0054, -4.3810,  ..., -4.5231, -4.6471, -4.8485],
            ...,
            [-4.6010, -4.5866, -4.4898,  ..., -4.3715, -4.2727, -4.4382],
            [-4.5806, -4.6248, -4.8934,  ..., -4.3190, -4.4208, -4.5099],
            [-4.5644, -4.7300, -4.6838,  ..., -4.3797, -4.3676, -4.6539]],
  
           [[-4.4077, -4.7504, -4.7537,  ..., -5.0152, -4.8749, -4.5994],
            [-4.4385, -4.5124, -4.1896,  ..., -4.9844, -4.6363, -4.3155],
            [-4.4624, -5.1129, -4.3522,  ..., -4.7806, -4.8280, -4.5345],
            ...,
            [-4.4559, -4.5407, -4.4826,  ..., -4.7122, -4.5267, -4.4566],
            [-4.4663, -4.6158, -4.7068,  ..., -4.8090, -4.7862, -4.8100],
            [-4.6702, -4.4927, -4.7266,  ..., -4.6503, -4.2228, -4.4610]],
  
           [[-4.3861, -4.3784, -4.7552,  ..., -4.5204, -4.6573, -4.691

In [ ]:
for i in model.model(img):
    for j in i:
        print(j.shape)

In [ ]:
for i in model.model(img):
    print(len(i))
    for j in i:
        print(j.shape)

In [ ]:
from torchinfo import summary

img = torch.randn(1, 3, 224, 224)
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=256)

x = img
model.eval()


for name, child in model.named_children():
    print(name, end="\t")
    if name == "avgpool":
        x = x.permute(0, 2, 1)
    if name == "head":
        x = x.squeeze(-1)
    print(x.shape, end="\t")
    x = eval(f"model.{name}(x)")
    print(x.shape)

assert (x == model(img)).all()

In [ ]:
import os
import sys

if "projects" not in os.getcwd():
  !git clone --depth 1 https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch
  os.chdir('Yet-Another-EfficientDet-Pytorch')
  sys.path.append('.')
else:
  !git pull

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
from torchinfo import summary

img = torch.randn(1, 3, 224, 224)
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=0)
summary(model, input_data=img, device="cpu")

## Dataset


### Train dataset

In [7]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TRAIN_CSV_PATH, names=range(4), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[3] = self.df_data[3].fillna("")
        self.df_data[2] += self.df_data[3]
        self.df_data = self.df_data.drop(3, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "label", 2: "caption"}, axis=1).dropna()

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Test dataset

In [8]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[2] = self.df_data[2].fillna("")
        self.df_data[1] += self.df_data[2]
        self.df_data = self.df_data.drop(2, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        caption = self.df_data.iloc[idx, 1]

        sample = {"caption": caption, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Preprocessing

### Dataset pre-transformations 

In [9]:
def one_hot_encode_labels(dataset):
    if "one_hot_encoded_labels" not in dataset.tags:
        dataset.df_data["label"] = dataset.df_data["label"].apply(
            lambda l: torch.nn.functional.one_hot(
                torch.tensor([int(i) - 1 if int(i) < 12 else int(i) - 2 for i in l.split(" ")]), 18
            )
            .sum(axis=0)
            .float()
        )

        dataset.tags.add("one_hot_encoded_labels")

## Modules


### Caption embedding

In [1]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
# lang_model = AutoModel.from_pretrained("prajjwal1/bert-mini")

tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-distilbert-base-cased")
lang_model = AutoModel.from_pretrained("sshleifer/tiny-distilbert-base-cased")

for name, param in lang_model.named_parameters():
    param.requires_grad_(False)

/home/ztan/miniconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


Downloading:   0%|          | 0.00/990 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/246k [00:00<?, ?B/s]

Some weights of the model checkpoint at sshleifer/tiny-distilbert-base-cased were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
lang_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 2, padding_idx=0)
    (position_embeddings): Embedding(512, 2)
    (token_type_embeddings): Embedding(2, 2)
    (LayerNorm): LayerNorm((2,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=2, out_features=2, bias=True)
            (key): Linear(in_features=2, out_features=2, bias=True)
            (value): Linear(in_features=2, out_features=2, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=2, out_features=2, bias=True)
            (LayerNorm): LayerNorm((2,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (inter

### Pretrained model surgery

In [11]:
from efficientnet_pytorch import EfficientNet

# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
# eff_net = EfficientNet.from_pretrained("efficientnet-b2")

swin = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=256)


class SWIN(nn.Module):
    def __init__(self, word_dim=256):
        super().__init__()
        self.model = timm.create_model(
            "swin_tiny_patch4_window7_224", pretrained=True, num_classes=word_dim
        )

    def forward(self, x):
        vec = None
        out = None
        for name, child in self.model.named_children():
            if name == "avgpool":
                vec = self.model.head(x)
                out = self.model.avgpool(x.permute(0, 2, 1)).squeeze(-1)
                return out, vec
            x = eval(f"self.model.{name}(x)")


# features = Flattened(eff_net)
features = SWIN(256)
for name, param in features.named_parameters():
    param.requires_grad_(False)
#     print(name) # layers.3.blocks.1


In [12]:
from torchinfo import summary

img = torch.randn(1,3,224,224)
# model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=256)

x = img
# model.eval()


#     print(x.shape)

# features = Flattened(model)

# assert (x == model(img)).all()
features(img)[0].shape, features(img)[1].shape

(torch.Size([1, 768]), torch.Size([1, 49, 256]))

### Combined model

In [ ]:
NUM_ClASSES = 18
import math

import torch.nn as nn

img_dim = 768
word_dim = list(lang_model.pooler.parameters())[-1].shape[0]


class SASGA(nn.Module):
    def __init__(self, word_dim=word_dim):
        super().__init__()
        self.layer_norm_1 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)
        self.layer_norm_2 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)

        self.guided_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim, dropout=0.1, activation="relu"
        )
        self.guided_attention = nn.TransformerDecoder(
            self.guided_attention_layer, num_layers=1, norm=self.layer_norm_1
        )
        self.self_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim, dropout=0.1, activation="relu"
        )
        self.self_attention = nn.TransformerDecoder(
            self.self_attention_layer, num_layers=1, norm=self.layer_norm_2
        )

    def forward(self, x, y, y_mask):
        # guide x by y
        x = self.self_attention_layer(x, x)
        x = self.guided_attention(x, y, memory_key_padding_mask=y_mask)
        return x


class Combined_Model(torch.nn.Module):
    def __init__(
        self, visual_features, lang_model, word_dim=word_dim, img_dim=img_dim, num_decoder_layers=1
    ):
        super().__init__()
        self.visual_features = visual_features
        self.lang_model = lang_model
        self.img_dim = img_dim
        self.word_dim = word_dim

        self.bottle_neck = nn.Conv2d(in_channels=img_dim, out_channels=word_dim, kernel_size=1)
        self.layer_norm = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)

        # SA-GA ##############################
        self.SASGA = SASGA(word_dim)
        # SA-GA ##############################

        self.output = nn.Linear(word_dim, NUM_ClASSES)
        self.activation = nn.ReLU()
        self.proj_all = nn.Linear(img_dim + word_dim * 2, word_dim)

    def forward(self, images, inputs):  # requires tokenized captions
        lang_outs = self.lang_model(**inputs)
        last_hidden_state = lang_outs["last_hidden_state"].permute(1, 0, 2)  # seq_len,
        pooler_out = lang_outs["pooler_output"]
        mask = inputs.attention_mask == 0

        images, image_query = self.visual_features(images)
#         image_query = intermediate_outputs["_conv_head"]
#         image_query = self.bottle_neck(image_query)

        # no param attention:
        #         image_query = image_query.view(-1, self.word_dim, 49)
        #         last_hidden_state = lang_outs["last_hidden_state"] # batch, seq_len, word_dim

        #         attention_weights = torch.bmm(last_hidden_state, image_query) # batch, seq_len, 49
        #         att_out = torch.bmm(attention_weights.permute(0,2,1), last_hidden_state).mean(1)
        #         att_out = self.layer_norm(att_out+image_query.mean(-1))

        #####################################################
        # self attention
        # image_query = image_query.view(-1, self.word_dim, 49).permute(2, 0, 1)
        # att_out = self.self_attention_layer(image_query, image_query)

        # #         # guided attention
        # att_out = self.guided_attention(att_out, last_hidden_state, memory_key_padding_mask=mask)
        # att_out = att_out.permute(1, 2, 0)  # 49, batch, word_dim -> batch, word_dim, 49
        # att_out = att_out.mean(-1)  # -> batch, word_dim
        #####################################################
        image_query = image_query.permute(1, 0, 2)

        att_out = self.SASGA(image_query, last_hidden_state, mask).mean(0).squeeze(0)
#         print(att_out.shape)
        out = torch.cat([images, att_out, pooler_out], dim=-1)
        out = self.proj_all(out)
        out = self.activation(out)
        out = self.output(out)
        return out

In [ ]:
# for name, _ in model.module_list.named_parameters():
#     print(name, _.requires_grad)

## Transforms and training support

In [13]:
class FieldTransform(object):
    def __init__(self, field, transform):
        self.field = field
        self.transform = transform

    def __call__(self, sample):
        sample[self.field] = self.transform(sample[self.field])
        return sample

## Experiments

In [14]:
def train_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["label"] = torch.stack(X["label"])
    X["image"] = torch.stack(X["image"])

    return X


normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms = torchvision.transforms.Compose(
    [
        FieldTransform("image", torchvision.transforms.Resize((224, 224))),
        FieldTransform("image", torchvision.transforms.ToTensor()),
        FieldTransform("image", normalize),
    ]
)

In [15]:
train_data = TrainDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(train_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
trainds, valds = random_split(train_data, [27000, 3000])

train_dl = DataLoader(
    trainds, batch_size=32, shuffle=True, collate_fn=train_collate_fn, num_workers=24
)

val_dl = DataLoader(valds, batch_size=8, shuffle=False, collate_fn=train_collate_fn, num_workers=24)

# model = Combined().to(DEVICE)


# model = Combined_Model(features, lang_model, num_decoder_layers=1).to(device)
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=18)
model.to(device)

from sam import SAM

# base_optimizer = torch.optim.Adam  # define an optimizer for the "sharpness-aware" update
# optimizer = SAM(model.parameters(), base_optimizer, lr=5e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = torch.nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=5,
    threshold=0.0001,
    threshold_mode="rel",
    cooldown=5,
    min_lr=0,
    eps=1e-08,
    verbose=False,
)

In [16]:
# for name, param in model.module_list.named_parameters():
#     print(name)
# raise ValueError

In [17]:
# torch.save(model, "test_v8.pt") #check model size

In [18]:
import os
from copy import deepcopy

from sklearn.metrics import f1_score
from tqdm import tqdm

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


val_mean_f1 = []
val_sample_f1 = []
model_in_memory = {}

scaler = torch.cuda.amp.GradScaler()
for epoch in range(50):
    train_loss = []
    train_outs = []
    train_lables = []
    #     model.train()

    #     fine tune CNN for 5 epoches
#     if epoch == 0:
#         finetune_param = False
#         for name, param in model.visual_features.named_parameters():
#             if "layers.3.blocks.1" in name:
#                 finetune_param = True
#             if finetune_param:
#                 print(name)
#                 param.requires_grad_(True)

    if epoch == 0:
#         finetune_param = False
#         for name, param in model.lang_model.named_parameters():
#             if "pooler" in name:
#                 finetune_param = True
#             if finetune_param:
#                 print(name)
#                 param.requires_grad_(True)
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name)

    for i, batch in enumerate(tqdm(train_dl)):
        optimizer.zero_grad()
#         captions = batch["caption"]
        images = batch["image"].to(device)
        labels = batch["label"].to(device)
#         inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        #         raise ValueError

        with torch.cuda.amp.autocast():
        # first second step
#             predictions = model(images, inputs)
            predictions = model(images)
            loss = criterion(predictions, labels)
#         loss.backward()
#         optimizer.step()
#         optimizer.first_step(zero_grad=True)

        #         with torch.cuda.amp.autocast():
        # second step
#         predictions = model(images, inputs)
#         loss = criterion(predictions, labels)
#         loss.backward()
#         optimizer.second_step(zero_grad=True)
        #
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss.append(loss.item())

    val_loss = []
    val_outs = []
    val_labels = []

    model.eval()
    for i, batch in enumerate(val_dl):

#         captions = batch["caption"]
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)
#         inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        #         predictions = model(images, captions)
        #         with torch.cuda.amp.autocast():
#         predictions = model(images, inputs)
        predictions = model(images)
        loss = criterion(predictions, labels)
        val_loss.append(loss.item())
        val_outs.append(predictions.detach().cpu().numpy())
        val_labels.append(labels.detach().cpu().numpy())
    val_labels = np.vstack(val_labels)
    val_outs = np.vstack(val_outs)
    mean_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="micro")  # mean f1
    sample_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="samples")  # mean f1

    cur_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch:{epoch}, train/val loss:{round(np.mean(train_loss),7),round(np.mean(val_loss),7)},sample/mean f1:{round(sample_f1, 7), round(mean_f1, 7)}, lr:{cur_lr}"
    )
    scheduler.step(np.mean(val_loss))

    if len(val_sample_f1) == 0 or sample_f1 > max(val_sample_f1):
        print("saving best")
        try:
            del model_in_memory["best"]
        except:
            pass
        torch.cuda.empty_cache()
        model.to("cpu")
        model_in_memory["best"] = deepcopy(model.state_dict())
        model.to(device)
    if epoch % 10 == 0:
        torch.save(model_in_memory["best"], "test_v8.pt")

    val_mean_f1.append(mean_f1)
    val_sample_f1.append(sample_f1)

  0%|          | 0/844 [00:00<?, ?it/s]

patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.1.norm1.weight
layers.0.blocks.1.norm1.bias
layers.0.blocks.1.attn.relative_position_bias_table
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.qkv.bias
layers.0.blocks.1.attn.proj.weight
layers.0.blocks.1.attn.proj.bias
layers.0.blocks.1.norm2.weight
layers.0.blocks.1.norm2.bias
layers.0.blocks.1.mlp.fc1.weight
layers.0.blocks.1.mlp.fc1.bias
layers.0.blocks.1.mlp.fc2.weight
layers.0.blocks.1.mlp.fc2.bias
layers.0.downsample.reduction.weight
lay

100%|██████████| 844/844 [02:14<00:00,  6.29it/s]


Epoch:0, train/val loss:(0.2225636, 0.2015422),sample/mean f1:(0.647846, 0.6024285), lr:0.01
saving best


  0%|          | 0/844 [00:00<?, ?it/s]

...

Epoch:4, train/val loss:(nan, 0.2007779),sample/mean f1:(0.647846, 0.6024285), lr:0.01


  0%|          | 0/844 [00:00<?, ?it/s]

Epoch:5, train/val loss:(nan, 0.2007779),sample/mean f1:(0.647846, 0.6024285), lr:0.01


100%|██████████| 844/844 [02:00<00:00,  6.99it/s]


KeyboardInterrupt: 

###################

In [ ]:
raise ValueError
# (finetune last 3 layer (15-17)) + (SAM, lr 0.001)
# b0 + bert tiny + no attention = 85
# b0 + bert tiny + SAGA * 1 = 85 ...
# b0 + bert tiny + bmm (attention but no weights) at last conv filters = fails..?! (lr too high )
# b0 (13-) + bert tiny + bmm (attention but no weights) = 85
# b0 + bert tiny (both no finetune) + SAGA * 1 (5e-5 lr)= 86.40
# b2 + bert mini (both no finetune) + SAGA * 1 (5e-5 lr)= 86.3
# b2 + bert mini (finetune from 21/3 attention) + SAGA * 1 (5e-5 lr)= 87.4..
# swin = 89.2
# swin by itself: doesnt work

In [ ]:
outs = lang_model(**inputs)
last_hidden_state = outs.last_hidden_state
pooler_output = outs.pooler_output

In [ ]:
image_feats, all_outs = features(images)

In [ ]:
all_outs.keys()

In [ ]:
all_outs["_conv_head"].shape

In [ ]:
comp.shape, last_hidden_state.shape

In [ ]:
bottle_neck = nn.Conv2d(in_channels=1408, out_channels=256, kernel_size=1).to(device)
norm = nn.LayerNorm(256, eps=1e-05, elementwise_affine=True)
decoder_layer = nn.TransformerDecoderLayer(
    256, 8, dim_feedforward=1024, dropout=0.1, activation="relu"
)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=1, norm=norm).to(device)

comp = bottle_neck(all_outs["_conv_head"]).view(-1, 256, 49).permute(2, 0, 1)
decoder(comp, last_hidden_state, memory_key_padding_mask=(inputs.attention_mask == 0)).shape

In [ ]:
import torch
import torch.nn as nn

images = torch.randn(1, 3, 224, 224)

In [ ]:
from efficientnet_pytorch import EfficientNet

eff_net = EfficientNet.from_pretrained("efficientnet-b2")

for name, param in eff_net.named_parameters():
    param.requires_grad_(False)


class Flattened(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in pretrained_model.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

    def forward(self, img_batch):
        all_outs = {}
        x = img_batch
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            else:
                x = module(x)

            all_outs[self.module_names[idx]] = x
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)
        return x, all_outs


eff_net.eval()
eff_net
f = Flattened(eff_net)
x, all_outs = f(images)

In [ ]:
activation = {}


def get_activation(name):
    def hook(eff_net, input, output):
        activation[name] = output

    return hook


for name, child in eff_net.named_children():
    if isinstance(child, nn.ModuleList):
        for idx, i in enumerate(child):
            i.register_forward_hook(get_activation(f"{name}_{str(idx)}"))
    else:
        child.register_forward_hook(get_activation(name))


output = eff_net(images)

In [ ]:
for key in activation.keys():
    if key == "_swish":
        continue
    print(key)
    print((all_outs[key] == activation[key]).all())

In [ ]:
# eff_net.to("cuda")
# f.to("cuda")

# %timeit -n10 -r5 eff_net(images.cuda())
# %timeit -n10 -r5 f(images.cuda())

In [ ]:
inputs.input_ids.shape

In [ ]:
from torchinfo import summary

batch_size = 1
summary(f, input_size=(batch_size, 3, 224, 224))

In [ ]:
for k, v in all_outs.items():
    print(k, v.shape)

In [ ]:
208 * 3 + 352 * 2

# correction

In [ ]:
# model_name = "test.pt"
# model = LUKE_CRF(tag_to_ix, lang_model=lang_model, hidden_dim=1024)
model = Combined_Model(features, lang_model)
model.load_state_dict(torch.load("test.pt"))
model.to(device)
# model = torch.load(model_name)

In [ ]:
# import numpy as np
# from sklearn.metrics import roc_curve, precision_recall_curve

# model = torch.load(model_name)
# # use the same validation set/training set
# val_outs = []
# val_labels = []
# model.eval()
# for i, batch in enumerate(val_dl):

#     captions = batch["caption"]
#     images = batch["image"].to(DEVICE)
#     labels = batch["label"].to(DEVICE)

#     inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
#     input_ids = inputs["input_ids"].to(device)
#     attention_mask = inputs["attention_mask"].to(device)

#     # predictions = model(images, captions)
#     predictions = model(images, input_ids, attention_mask)

#     val_outs.append(predictions.detach().cpu().numpy())
#     val_labels.append(labels.detach().cpu().numpy())
# val_labels = np.vstack(val_labels)
# val_outs = np.vstack(val_outs)

# best_thresholds = np.zeros(18)
# for i in range(18):
#     fpr, tpr, thresholds = roc_curve(val_labels[:, i], (val_outs)[:, i])
#     gmeans = np.sqrt(tpr * (1 - fpr))
#     ix = np.argmax(gmeans)
#     best_thresholds[i] = thresholds[ix]

In [ ]:
# cheat
class CheatDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pd.read_csv("test_cheat.csv", names=range(3), skiprows=1)[[0, 2, 1]].dropna()
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data = self.df_data.rename({0: "image", 2: "label", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample


cheat_data = CheatDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(cheat_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
cheat_dl = DataLoader(
    cheat_data, batch_size=32, shuffle=False, collate_fn=train_collate_fn, num_workers=24
)

from sklearn.metrics import precision_recall_curve

# use the same validation set/training set
val_outs = []
val_labels = []
model.eval()
for i, batch in enumerate(cheat_dl):

    captions = batch["caption"]
    images = batch["image"].to(device)
    labels = batch["label"].to(device)
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    # predictions = model(images, captions)
    predictions = model(images, inputs)

    val_outs.append(predictions.detach().cpu().numpy())
    val_labels.append(labels.detach().cpu().numpy())
val_labels = np.vstack(val_labels)
val_outs = np.vstack(val_outs)


best_thresholds = np.zeros(18)
for i in range(18):
    fpr, tpr, thresholds = precision, recall, thresholds = precision_recall_curve(
        val_labels[:, i], (val_outs)[:, i]
    )
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    best_thresholds[i] = thresholds[ix]

print(best_thresholds)
np.save("best_thresholds", best_thresholds)

# generate test labels

In [ ]:
# stopper for run all
raise ValueError

In [ ]:
def test_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["image"] = torch.stack(X["image"])

    return X


test_data = TestDataset(transform=transforms)
test_dl = torch.utils.data.DataLoader(
    test_data, batch_size=16, shuffle=False, collate_fn=test_collate_fn, num_workers=0,
)


# save predictions
model.eval()
test_preds = []
for i, batch in enumerate(test_dl):
    captions = batch["caption"]
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    predictions = model(images, inputs)
    test_preds.append(predictions.detach().cpu().numpy())

test_preds = np.vstack(test_preds)


def out_logits_to_preds(logits, best_thresholds):
    labels = []
    logits = (logits - np.zeros_like(best_thresholds)) > 0
    lables_available = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19])
    for i in range(logits.shape[0]):
        labels.append(list(lables_available[logits[i]]))
    return labels


# lables_available[(test_preds[:10] > 0)]
# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}

test_labels = out_logits_to_preds(test_preds, best_thresholds)

df_test = pd.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
test_labels_str = [" ".join([str(i) for i in labels]) for labels in test_labels]
df_test["Labels"] = test_labels_str
df_test.rename({0: "ImageID"}, axis=1, inplace=True)

df_test[["ImageID", "Labels"]].to_csv("test_predictions.csv", index=False)